In [ ]:
# %pip install xlrd
import pandas as pd
import numpy as np

df = pd.read_excel("/Users/snow/Documents/IRON HACK COURSE/Week 2/Mini Project/Week-2-Mini-Project---Shark-Attacks/GSAF5.xls")

# Now we want to clean the dataframe a bit
# First lets have a look at the Dataframe
print(df.shape, '\n')
print(df.nunique(), '\n')

# Then we want to standardize the column names
df.columns = df.columns.str.lower().str.replace(" ","_").str.replace(":", "")
df.rename(columns={"species_":"species"}, inplace=True)

# We can see that there are some weird values in the 'Type' column such as 'Boat', 'Questionable ', 'Unconfirmed'
# We want to merge these values into a single category called 'Under investigation'

# First, we drop the rows where 'Type' and 'Sex' are null
df.dropna(subset=["sex", "type"], inplace=True)

# Then, we replace the values in the 'Type' column
def replace_type(value):
    if isinstance(value, str):
        # Remove white spaces and compare lower-case
        value_clean = value.strip().lower()
        valid_types = ['unprovoked', 'provoked']
        if value_clean not in valid_types:
            return "Under investigation"
        else:
            # Return the standardized value (capitalize first letter)
            return value_clean.capitalize()
    return value

df["type"] = df["type"].apply(replace_type)

# Now we can check the value counts again
print(df["type"].value_counts(), '\n')

# Now we want to clean the 'sex' column in the same way
def replace_sex(value):
    if isinstance(value, str):
        if value[0].lower() == "f":
            return "F"
        else:
            return "M"

df["sex"] = df["sex"].apply(replace_sex)
print(df["sex"].value_counts(), '\n')
print(df['sex'].unique(), '\n')

df['time'].value_counts()
df = df.drop(df.columns[[7, 14, 15, 16, 17, 18, 19, 20, 21, 22]], axis=1)

df.head()



(7045, 23) 

Date              6089
Year               261
Type                13
Country            247
State              937
Location          4595
Activity          1606
Name              5770
Sex                 10
Age                250
Injury            4162
Fatal Y/N           12
Time               460
Species           1724
Source            5384
pdf               6789
href formula      6784
href              6776
Case Number       6777
Case Number.1     6775
original order    6797
Unnamed: 21          1
Unnamed: 22          2
dtype: int64 

type
Unprovoked             5032
Under investigation     834
Provoked                582
Name: count, dtype: int64 

sex
M    5645
F     803
Name: count, dtype: int64 

['M' 'F'] 



,date,year,type,country,state,location,activity,sex,age,injury,fatal_y/n,time,species
0,6th September,2025.0,Unprovoked,Australia,NSW,Long Reef Sydney,Surfing,M,57,Both legs and arm severed,Y,0930hrs,Great White Shark
1,1st September,2025.0,Unprovoked,USA,Florida,Horseshoe reef Key Largo,Snorkeling,M,8,Bite to leg,N,1524hrs,Not stated
2,30th August,2025.0,Unprovoked,USA,Texas,Galveston,Swimming,F,8,Bite to leg,N,Not stated,Not stated
3,18th August,2025.0,Unprovoked,Australia,NSW,Cabarita Beach,Surfing,M,?,None sustained board severly damaged,N,0730hrs,5m (16.5ft) Great White
4,17th August,2025.0,Unprovoked,Bahamas,Atlantic Ocean near Big Grand Cay,North of Grand Bahama near Freeport,Spearfishing,M,63,Severe injuries no detail,N,1300hrs,Undetermined
